# 进程间的通信  
1. 管道  
2. 进程信号量  
3. 共享内存  
4. 映射内存  
5. 消息队列  
6. 套接字  

---  
## 1.管道（pipe）
函数原型：
```C
#include<unistd.h>
int filedes[2];
//返回值：成功，返回0，否则返回-1。
int pipe(int filedes[2]);
```

管道是基于fork()机制，所以只能用于拥有相同祖先的两个子进程或用于父子进程之间的通信。  
命名管道（FIFO）方式可以使无亲缘关系的进程之间通信。  
下面例子中，创建一对父子进程，子进程负责从文件中读取信息，父进程负责向文件中写内容。  
> **疑问：**创建pipe需要一个int fds[2]，这是一个空的数组，为什么可以进行文件读写呢？  Linux文件描述符   
在<unistd.h>文件中，有C语言的三个标准文件描述符  

```C
/* Standard file descriptors.  */
#define	STDIN_FILENO	0	/* Standard input.  */
#define	STDOUT_FILENO   1	/* Standard output.  */
#define	STDERR_FILENO	2	/* Standard error output.  */
```  

进程之间通过管道（pipe）进行通信的例子： 

```C++
#include "pipe-communication.h"
void my_pipe_communication_main(){
   int fds[2];          //in pipe, fds[0] is the signal for reading, fds[1] is the signal for writing
   pipe(fds);           //create pipe
   
    /*fork方法创建一个子进程，它有两个返回值，返回的是当前进程的子进程id。
     *因此如果当前进程是父进程的话pid应大于0；
     *如果是子进程的话，由于子进程没有子进程，因此pid值为0。这里通过if-else来判断当前进程为子进程或父进程。
     */
   pid_t pid = fork();  //create subprocess

   if(pid == 0){        //if process is subprocess
       close(fds[1]);   //close write pipe
       FILE * stream = fdopen(fds[0], "r");
       Read(stream);
       close(fds[0]);
   }else if(pid > 0){   //if process is parent process
        char buf[buf_size];
        for(int i = 0; i < buf_size-2; i ++){
            buf[i] = 'A' + i % 26;
        }
        buf[buf_size-1] = buf[buf_size-2] = '\0';
        close(fds[0]);
        FILE * stream = fdopen(fds[1],"w");
        Write(buf, 3, stream);
        close(fds[1]);
   }
}

void Write(const char * msg, int count, FILE * stream){
    for(; count > 0; -- count){
        fprintf(stream, "%s\n", msg);
        //fflush()会强迫将缓冲区内的数据写回参数stream 指定的文件中。
        fflush(stream);
        sleep(1);
    }
}

void Read(FILE * stream){
    char buf[buf_size];
    /*feof是C语言标准库函数，其原型在stdio.h中，其功能是检测流上的文件结束符，
      如果文件结束，则返回非0值，否则返回0（即，文件结束：返回非0值，文件未结束，返回0值）
     *ferror，在调用各种输入输出函数（如putc.getc.等）时，如果出现错误，除了函数返回值有所反映外，还可以用ferror函数检查。 
      如果ferror返回值为0（假），表示未出错。如果返回一个非零值，表示出错。
     *fgets，从文件结构体指针stream中读取数据，每次读取一行。
     */
    while (!feof(stream) && !ferror(stream) && fgets(buf, sizeof(buf),stream) != NULL){
        fprintf(stdout, "Data receive:\n");
        fputs(buf, stdout);
    }
}
```  

**管道重定向（dup()）**  
在之前父子进程通过管道通信的例子中，通过自定义的linux文件描述符fds进行文件的读写，下面的例子使用dup函数，将定义的fds[1]写文件描述符重定向到标准文件输出(STDIYT_FILENO)这个系统的标准文件描述符中。并使用execvp命令执行这个命令。  

```C
#include "pipe-dup.h"
#include "pipe-dup.h"
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <fcntl.h>
#include <sys/wait.h>

void my_pipe_dup_main(){
    int fds[2];
    pipe(fds);
    pid_t pid = fork();

    if(pid == (pid_t)0){                //subprocess write data
        close(fds[0]);
        dup2(fds[1], STDOUT_FILENO);    //dup the fds[1] to STDOUT_FILENO
        char* args[] = {"ls", "-la", "/root/Documents/", NULL};
        execvp(args[0], args);          //???
    }else{                              //parent process read data
        close(fds[1]);
        char buf[buf_size];
        FILE * stream = fdopen(fds[0], "r");
        fprintf(stdout, "Data receive:\n");
        while (!feof(stream) && !ferror(stream) && fgets(buf, sizeof(buf), stream) != NULL){
            fputs(buf, stdout);
        }
        close(0);
        //阻塞父进程，等待子进程结束
        waitpid(pid, NULL, 0);
    }
}
```

1)  wait()函数用于使父进程（也就是调用wait()的进程）阻塞，直到一个子进程结束或者该进程接收到了一个指定的信号为止。如果该父进程没有子进程或者它的子进程已经结束，则wait()函数就会立即返回。

2) waitpid()的作用和wait()一样，但它并不一定要等待第一个终止的子进程（它可以指定需要等待终止的子进程），它还有若干选项，如可提供一个非阻塞版本的 wait()功能，也能支持作业控制。实际上，wait()函数只是 waitpid()函数的一个特例，在Linux 内部实现 wait()函数时直接调用的就是waitpid()函数。  
```C
//函数原型
pid_t waitpid(pid_t pid, int * status, int options);  
```  

|参数|说明|
|:--|:--|
|pid|pid > 0，只等待ID为pid的子进程|
|pid|pid = -1，等待任何一个子进程（这时，和wait一样）|
|pid|pid = 0，等待组ID等于调用进程的组ID的任意一个子进程|
|pid|pid < -1，等待组ID等于pid的绝对值的任意一个子进程|
|status|是子进程退出时的状态|
|option|WNOHANG，pid指定的子进程没有结束，waitpid不阻塞，立即返回0|
|option|WUNTRACED，pid指定的进程被暂停，且暂停后没有返回过状态，返回其状态|
|option|0，阻塞父进程，等待子进程退出|  

